## Logic Based FizzBuzz Function [Software 1.0]

In [1]:
import pandas as pd

def fizzbuzz(n):
    
    # Logic Explanation
    if n % 3 == 0 and n % 5 == 0:
        return 'fizzbuzz'
    elif n % 3 == 0:
        return 'fizz'
    elif n % 5 == 0:
        return 'buzz'
    else:
        return 'other'

## Create Training and Testing Datasets in CSV Format

In [2]:
def createInputCSV(start,end,filename):
    
    # Why list in Python?
    inputData   = []
    outputData  = []
    
    # Why do we need training Data?
    for i in range(start,end):
        inputData.append(i)
        outputData.append(fizzbuzz(i))
    
    # Why Dataframe?
    # Dataframes are pandas objects, which themselves are enhanced version of 
    # numpy structured arrays
    # DataFrames are inherently multidimensional arrays with 
    # attached row and column labels, and added support for heterogeneous 
    # types and missing data.
    dataset = {}
    dataset["input"]  = inputData
    dataset["label"] = outputData
    
    # Writing to csv
    pd.DataFrame(dataset).to_csv(filename)
    
    print(filename, "Created!")

## Processing Input and Label Data

In [3]:
def processData(dataset):
    """Processes input data and label.
    
    Parameters
    ---------
        dataset : pandas dataframe
            Dataframe containing data and labels

    Returns
    -------
        processedData : list
        processedLabel :  list
    """
    # Why do we have to process?
    # Preprocess input data to make it compatible for Neural Network
    data   = dataset['input'].values
    labels = dataset['label'].values
    
    processedData  = encodeData(data)
    processedLabel = encodeLabel(labels)
    
    return processedData, processedLabel

In [4]:
def encodeData(data):
    """Converts a number to its binary reporesentaiton
    """
    processedData = []
    
    for dataInstance in data:
        
        # Why do we have number 10?
        processedData.append([dataInstance >> d & 1 for d in range(10)])
    
    return np.array(processedData)

In [5]:
from keras.utils import np_utils
import os

# TF_CPP_MIN_LOG_LEVEL : Tensorflow environment variable
# Default: to 0 (all logs shown), 
# 1 to filter out INFO logs, 
# 2 to additionally filter out WARNING logs, 
# 3 to additionally filter out ERROR logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def encodeLabel(labels):
    """
    """
    processedLabel = []
    
    for labelInstance in labels:
        if(labelInstance == "fizzbuzz"):
            # Fizzbuzz
            processedLabel.append([3])
        elif(labelInstance == "fizz"):
            # Fizz
            processedLabel.append([1])
        elif(labelInstance == "buzz"):
            # Buzz
            processedLabel.append([2])
        else:
            # Other
            processedLabel.append([0])
    
    return np_utils.to_categorical(np.array(processedLabel),4)

Using TensorFlow backend.
/Users/ameenkhan/dev/projects/Machine-Learning/1/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Model Definition

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, TensorBoard

import numpy as np

input_size = 10 # 2^10 > 1000
drop_out = 0.2 # adjusting for overfitting

first_dense_layer_nodes  = 512 # first hidden layer
# Note : hidden layer shouldn't be too absurd, 
# not to large (overfitting)
# not to small
second_dense_layer_nodes = 4 # output layer

# This is a Sequential model, 2 layer deep
# 1 hidden layer, 1 output layer

def get_model():
    
    # Why do we need a model?
    # 'Model' is a data strucuture used by keras, a way to 
    # oragnize the layers.
    
    # Why use Dense layer and then activation?
    # Dense layer means fully connected layer.
    
    # Why use sequential model with layers?
    model = Sequential()
    
    # First Hidden Layer
    # Input dimensionality 900*10
    # reason : 900 numbers (101-1000), 10 bits each
    # Weight of first hidden layer = 10 * 256 (10 bits, 256 units in the first layer)
    # Dimensions from first layer = 900 * 256
    model.add(Dense(first_dense_layer_nodes, input_dim=input_size))
    
    
    # Activation Layer
    # Activation : Fucntions sigmoid/ relu 
    # Why Relu :relu is also used in regularization
    # Why not sigmoid?
    model.add(Activation('relu'))
    
    # Why dropout?
    # Dropout is a technique where randomly selected neurons are ignored during training.
    # The effect is that the network becomes less sensitive to the specific weights of neurons. 
    # This in turn results in a network that is capable of better generalization and is less likely to 
    # overfit the training data.
    model.add(Dropout(drop_out))
    
    # Second Layer, Output Layer
    model.add(Dense(second_dense_layer_nodes))
    model.add(Activation('softmax'))
    # Why Softmax?
    
    model.summary()
    
    
    # Experiment with optimizers: 
    # SGD, adagrad, adam etc.
    
    
    # Loss function
    # Cross Entropy: KL Divergence 
    # Why use categorical_crossentropy?
    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# <font color='blue'>Creating Training and Testing Datafiles</font>

In [7]:
# Create datafiles
createInputCSV(101,1001,'training.csv')
createInputCSV(1,101,'testing.csv')

training.csv Created!
testing.csv Created!


# <font color='blue'>Creating Model</font>

In [8]:
model = get_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               5632      
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 2052      
_________________________________________________________________
activation_2 (Activation)    (None, 4)                 0         
Total params: 7,684
Trainable params: 7,684
Non-trainable params: 0
_________________________________________________________________


# <font color = blue>Run Model</font>

In [9]:
validation_data_split = 0.2 # trying to tackle the problem of overfitting
# difference between validation to testing
# after every epoch we validate
# validation loss and training loss
# validation avoids over fitting

# cros validation: validation data keeps rotating


num_epochs = 10000 # experiment on this
model_batch_size = 128
# minibatch? : between the 2 approaches:
# fitting whole data into the model, 

tb_batch_size = 32
early_patience = 100

tensorboard_cb   = TensorBoard(log_dir='logs', batch_size= tb_batch_size, write_graph= True)
earlystopping_cb = EarlyStopping(monitor='val_loss', verbose=1, patience=early_patience, mode='min')

# Read Dataset
dataset = pd.read_csv('training.csv')

# Process Dataset
processedData, processedLabel = processData(dataset)
history = model.fit(processedData
                    , processedLabel
                    , validation_split=validation_data_split
                    , epochs=num_epochs
                    , batch_size=model_batch_size
                    , callbacks = [tensorboard_cb,earlystopping_cb]
                   )

Train on 720 samples, validate on 180 samples
Epoch 1/10000
720/720 [==============================] - 0s 243us/step - loss: 1.3074 - acc: 0.4264 - val_loss: 1.1591 - val_acc: 0.5333
Epoch 2/10000
720/720 [==============================] - 0s 17us/step - loss: 1.1682 - acc: 0.5333 - val_loss: 1.1435 - val_acc: 0.5333
Epoch 3/10000
720/720 [==============================] - 0s 18us/step - loss: 1.1578 - acc: 0.5333 - val_loss: 1.1456 - val_acc: 0.5333
Epoch 4/10000
720/720 [==============================] - 0s 19us/step - loss: 1.1436 - acc: 0.5333 - val_loss: 1.1578 - val_acc: 0.5333
Epoch 5/10000
720/720 [==============================] - 0s 15us/step - loss: 1.1513 - acc: 0.5333 - val_loss: 1.1461 - val_acc: 0.5333
Epoch 6/10000
720/720 [==============================] - 0s 15us/step - loss: 1.1461 - acc: 0.5333 - val_loss: 1.1479 - val_acc: 0.5333
Epoch 7/10000
720/720 [==============================] - 0s 18us/step - loss: 1.1432 - acc: 0.5333 - val_loss: 1.1504 - val_acc: 0.5333
E

720/720 [==============================] - 0s 15us/step - loss: 1.0001 - acc: 0.5458 - val_loss: 1.1021 - val_acc: 0.5389
Epoch 61/10000
720/720 [==============================] - 0s 14us/step - loss: 0.9960 - acc: 0.5472 - val_loss: 1.1013 - val_acc: 0.5389
Epoch 62/10000
720/720 [==============================] - 0s 14us/step - loss: 1.0003 - acc: 0.5458 - val_loss: 1.0965 - val_acc: 0.5389
Epoch 63/10000
720/720 [==============================] - 0s 13us/step - loss: 0.9956 - acc: 0.5556 - val_loss: 1.0985 - val_acc: 0.5333
Epoch 64/10000
720/720 [==============================] - 0s 14us/step - loss: 0.9877 - acc: 0.5569 - val_loss: 1.0939 - val_acc: 0.5389
Epoch 65/10000
720/720 [==============================] - 0s 14us/step - loss: 0.9823 - acc: 0.5556 - val_loss: 1.0930 - val_acc: 0.5444
Epoch 66/10000
720/720 [==============================] - 0s 15us/step - loss: 0.9812 - acc: 0.5694 - val_loss: 1.0937 - val_acc: 0.5500
Epoch 67/10000
720/720 [==============================] 

Epoch 120/10000
720/720 [==============================] - 0s 15us/step - loss: 0.8000 - acc: 0.6569 - val_loss: 0.9779 - val_acc: 0.5667
Epoch 121/10000
720/720 [==============================] - 0s 13us/step - loss: 0.7984 - acc: 0.6500 - val_loss: 0.9778 - val_acc: 0.5778
Epoch 122/10000
720/720 [==============================] - 0s 13us/step - loss: 0.7899 - acc: 0.6750 - val_loss: 0.9713 - val_acc: 0.5722
Epoch 123/10000
720/720 [==============================] - 0s 15us/step - loss: 0.7794 - acc: 0.6667 - val_loss: 0.9696 - val_acc: 0.5722
Epoch 124/10000
720/720 [==============================] - 0s 15us/step - loss: 0.7811 - acc: 0.6778 - val_loss: 0.9696 - val_acc: 0.5778
Epoch 125/10000
720/720 [==============================] - 0s 14us/step - loss: 0.7815 - acc: 0.6722 - val_loss: 0.9686 - val_acc: 0.5778
Epoch 126/10000
720/720 [==============================] - 0s 14us/step - loss: 0.7748 - acc: 0.6819 - val_loss: 0.9638 - val_acc: 0.5833
Epoch 127/10000
720/720 [=========

720/720 [==============================] - 0s 15us/step - loss: 0.6069 - acc: 0.7986 - val_loss: 0.8374 - val_acc: 0.7056
Epoch 180/10000
720/720 [==============================] - 0s 13us/step - loss: 0.5965 - acc: 0.8194 - val_loss: 0.8416 - val_acc: 0.6111
Epoch 181/10000
720/720 [==============================] - 0s 13us/step - loss: 0.6145 - acc: 0.7708 - val_loss: 0.8330 - val_acc: 0.6722
Epoch 182/10000
720/720 [==============================] - 0s 14us/step - loss: 0.5880 - acc: 0.8125 - val_loss: 0.8306 - val_acc: 0.6778
Epoch 183/10000
720/720 [==============================] - 0s 14us/step - loss: 0.5992 - acc: 0.8014 - val_loss: 0.8292 - val_acc: 0.6889
Epoch 184/10000
720/720 [==============================] - 0s 15us/step - loss: 0.6005 - acc: 0.8069 - val_loss: 0.8260 - val_acc: 0.6833
Epoch 185/10000
720/720 [==============================] - 0s 14us/step - loss: 0.5954 - acc: 0.7903 - val_loss: 0.8306 - val_acc: 0.7556
Epoch 186/10000
720/720 [=========================

720/720 [==============================] - 0s 14us/step - loss: 0.4886 - acc: 0.8736 - val_loss: 0.7485 - val_acc: 0.6500
Epoch 239/10000
720/720 [==============================] - 0s 14us/step - loss: 0.4684 - acc: 0.8597 - val_loss: 0.7205 - val_acc: 0.6889
Epoch 240/10000
720/720 [==============================] - 0s 15us/step - loss: 0.4687 - acc: 0.8667 - val_loss: 0.7446 - val_acc: 0.6389
Epoch 241/10000
720/720 [==============================] - 0s 13us/step - loss: 0.4515 - acc: 0.8639 - val_loss: 0.7127 - val_acc: 0.7556
Epoch 242/10000
720/720 [==============================] - 0s 14us/step - loss: 0.4659 - acc: 0.8681 - val_loss: 0.7065 - val_acc: 0.8333
Epoch 243/10000
720/720 [==============================] - 0s 16us/step - loss: 0.4594 - acc: 0.8750 - val_loss: 0.7070 - val_acc: 0.7611
Epoch 244/10000
720/720 [==============================] - 0s 14us/step - loss: 0.4776 - acc: 0.8625 - val_loss: 0.7171 - val_acc: 0.6944
Epoch 245/10000
720/720 [=========================

...

720/720 [==============================] - 0s 15us/step - loss: 0.2005 - acc: 0.9417 - val_loss: 0.5265 - val_acc: 0.8500
Epoch 534/10000
720/720 [==============================] - 0s 14us/step - loss: 0.2058 - acc: 0.9444 - val_loss: 0.5640 - val_acc: 0.7778
Epoch 535/10000
720/720 [==============================] - 0s 14us/step - loss: 0.2093 - acc: 0.9403 - val_loss: 0.4891 - val_acc: 0.8556
Epoch 536/10000
720/720 [==============================] - 0s 13us/step - loss: 0.2015 - acc: 0.9472 - val_loss: 0.5340 - val_acc: 0.8222
Epoch 537/10000
720/720 [==============================] - 0s 15us/step - loss: 0.2074 - acc: 0.9375 - val_loss: 0.5543 - val_acc: 0.8056
Epoch 538/10000
720/720 [==============================] - 0s 14us/step - loss: 0.2187 - acc: 0.9361 - val_loss: 0.5342 - val_acc: 0.8167
Epoch 539/10000
720/720 [==============================] - 0s 14us/step - loss: 0.1749 - acc: 0.9611 - val_loss: 0.5480 - val_acc: 0.8000
Epoch 540/10000
720/720 [=========================

720/720 [==============================] - 0s 14us/step - loss: 0.1947 - acc: 0.9458 - val_loss: 0.5204 - val_acc: 0.8111
Epoch 593/10000
720/720 [==============================] - 0s 14us/step - loss: 0.1811 - acc: 0.9542 - val_loss: 0.5032 - val_acc: 0.8556
Epoch 594/10000
720/720 [==============================] - 0s 13us/step - loss: 0.1834 - acc: 0.9569 - val_loss: 0.4891 - val_acc: 0.8778
Epoch 595/10000
720/720 [==============================] - 0s 13us/step - loss: 0.1628 - acc: 0.9597 - val_loss: 0.4724 - val_acc: 0.8889
Epoch 596/10000
720/720 [==============================] - 0s 13us/step - loss: 0.1908 - acc: 0.9389 - val_loss: 0.5870 - val_acc: 0.7722
Epoch 597/10000
720/720 [==============================] - 0s 14us/step - loss: 0.1741 - acc: 0.9514 - val_loss: 0.5376 - val_acc: 0.8056
Epoch 598/10000
720/720 [==============================] - 0s 13us/step - loss: 0.1751 - acc: 0.9528 - val_loss: 0.5789 - val_acc: 0.7833
Epoch 599/10000
720/720 [=========================

720/720 [==============================] - 0s 13us/step - loss: 0.1759 - acc: 0.9444 - val_loss: 0.4734 - val_acc: 0.8944
Epoch 652/10000
720/720 [==============================] - 0s 14us/step - loss: 0.1571 - acc: 0.9625 - val_loss: 0.4597 - val_acc: 0.8944
Epoch 653/10000
720/720 [==============================] - 0s 14us/step - loss: 0.1504 - acc: 0.9583 - val_loss: 0.5028 - val_acc: 0.8722
Epoch 654/10000
720/720 [==============================] - 0s 13us/step - loss: 0.1632 - acc: 0.9528 - val_loss: 0.4734 - val_acc: 0.8778
Epoch 655/10000
720/720 [==============================] - 0s 13us/step - loss: 0.1650 - acc: 0.9556 - val_loss: 0.5309 - val_acc: 0.8444
Epoch 656/10000
720/720 [==============================] - 0s 14us/step - loss: 0.1572 - acc: 0.9569 - val_loss: 0.4751 - val_acc: 0.8833
Epoch 657/10000
720/720 [==============================] - 0s 14us/step - loss: 0.1644 - acc: 0.9542 - val_loss: 0.5220 - val_acc: 0.8389
Epoch 658/10000
720/720 [=========================

720/720 [==============================] - 0s 14us/step - loss: 0.1617 - acc: 0.9514 - val_loss: 0.4547 - val_acc: 0.8889
Epoch 711/10000
720/720 [==============================] - 0s 14us/step - loss: 0.1495 - acc: 0.9583 - val_loss: 0.5512 - val_acc: 0.7944
Epoch 712/10000
720/720 [==============================] - 0s 14us/step - loss: 0.1590 - acc: 0.9542 - val_loss: 0.4546 - val_acc: 0.8944
Epoch 713/10000
720/720 [==============================] - 0s 15us/step - loss: 0.1377 - acc: 0.9625 - val_loss: 0.5575 - val_acc: 0.8111
Epoch 714/10000
720/720 [==============================] - 0s 13us/step - loss: 0.1530 - acc: 0.9583 - val_loss: 0.5425 - val_acc: 0.8389
Epoch 715/10000
720/720 [==============================] - 0s 15us/step - loss: 0.1397 - acc: 0.9556 - val_loss: 0.5404 - val_acc: 0.8444
Epoch 716/10000
720/720 [==============================] - 0s 15us/step - loss: 0.1542 - acc: 0.9514 - val_loss: 0.4859 - val_acc: 0.8833
Epoch 717/10000
720/720 [=========================

720/720 [==============================] - 0s 15us/step - loss: 0.1498 - acc: 0.9556 - val_loss: 0.5652 - val_acc: 0.8111
Epoch 770/10000
720/720 [==============================] - 0s 15us/step - loss: 0.1481 - acc: 0.9611 - val_loss: 0.5561 - val_acc: 0.8111
Epoch 771/10000
720/720 [==============================] - 0s 14us/step - loss: 0.1620 - acc: 0.9500 - val_loss: 0.5821 - val_acc: 0.8000
Epoch 772/10000
720/720 [==============================] - 0s 14us/step - loss: 0.1528 - acc: 0.9472 - val_loss: 0.4968 - val_acc: 0.8778
Epoch 773/10000
720/720 [==============================] - 0s 16us/step - loss: 0.1553 - acc: 0.9486 - val_loss: 0.4713 - val_acc: 0.8889
Epoch 774/10000
720/720 [==============================] - 0s 16us/step - loss: 0.1655 - acc: 0.9431 - val_loss: 0.4847 - val_acc: 0.8889
Epoch 775/10000
720/720 [==============================] - 0s 13us/step - loss: 0.1388 - acc: 0.9639 - val_loss: 0.5003 - val_acc: 0.8722
Epoch 776/10000
720/720 [=========================

# <font color = blue>Training and Validation Graphs</font>

In [10]:
df = pd.DataFrame(history.history)
df.plot(subplots=True, grid=True, figsize=(10,15))

array([<matplotlib.axes._subplots.AxesSubplot object at 0x1275d1828>,
      dtype=object)

# <font color = blue>Testing Accuracy [Software 2.0]</font>

In [11]:
def decodeLabel(encodedLabel):
    if encodedLabel == 0:
        return "other"
    elif encodedLabel == 1:
        return "fizz"
    elif encodedLabel == 2:
        return "buzz"
    elif encodedLabel == 3:
        return "fizzbuzz"

In [12]:
wrong   = 0
right   = 0

testData = pd.read_csv('testing.csv')

processedTestData  = encodeData(testData['input'].values)
processedTestLabel = encodeLabel(testData['label'].values)
predictedTestLabel = []

for i,j in zip(processedTestData,processedTestLabel):
    y = model.predict(np.array(i).reshape(-1,10))
    predictedTestLabel.append(decodeLabel(y.argmax()))
    
    if j.argmax() == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))

print("Testing Accuracy: " + str(right/(right+wrong)*100))

output = {}
output["input"] = testData['input']
output["label"] = testData['label']
output["predicted_label"] = predictedTestLabel

opdf = pd.DataFrame(output)
opdf.to_csv('output.csv')

Errors: 15  Correct :85
Testing Accuracy: 85.0
